<a href="https://colab.research.google.com/github/xxichlas/explore-detection-vocal/blob/main/Vocal_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu==2.7.0

In [ ]:
!pip install tensorflow-gpu==2.4.1

In [ ]:
!pip install tensorflow==2.3.0.

In [ ]:
%tensorflow_version 2.4.1

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# import tensorflow as tf
# from keras.layers import LayerNormalization
print(tf.__version__)

## Clone Git Vocal

In [ ]:
!git clone https://github.com/xxichlas/explore-detection-vocal.git

## **Cloning TFOD 2.0 Github**

In [ ]:
cd /content/explore-detection-vocal

In [ ]:
cd Tensorflow

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
pwd

In [ ]:
cd models/research

In [ ]:
pwd

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git


In [ ]:
cd cocoapi/PythonAPI

In [ ]:
!make

In [ ]:
cp -r pycocotools /content/explore-detection-vocal/Tensorflow/models/research

### Install the Object Detection API

In [ ]:
cd ..

In [ ]:
cd .. 

In [ ]:
pwd

In [ ]:
cd /content/models/research

In [ ]:
cp object_detection/packages/tf2/setup.py .

In [ ]:
!python -m pip install .

In [ ]:
# From within TensorFlow/models/research/
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
# Downgrade opencv
!pip uninstall opencv-python-headless==4.5.5.64
! pip install opencv-python-headless==4.5.2.52

## Setup Path 

In [ ]:
pwd

In [ ]:
cd ..

In [ ]:
cd ..

In [ ]:
cd ..

In [ ]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

## Create Label Map

In [ ]:
labels = [
    {'name':'A', 'id':1}, 
    {'name':'E', 'id':2},
    {'name':'I', 'id':3},
    {'name':'O', 'id':4},
    {'name':'U', 'id':5},
    ]

with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## Create TF Records

In [ ]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

## Copy Model Config to Training Folder


In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [ ]:
!mkdir {'Tensorflow/workspace/models/'+CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

## Update Config

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [ ]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [ ]:
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = 5
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

## Train Model

In [ ]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

In [ ]:
!python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=15000

In [ ]:
!zip -r /content/explore-detection-vocal/Tensorflow/workspace/models.zip /content/explore-detection-vocal/Tensorflow/workspace/models

In [ ]:
pwd

In [ ]:
!python /content/explore-detection-vocal/Tensorflow/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir /content/explore-detection-vocal/Tensorflow/workspace/models/my_ssd_mobnet \
    --output_directory /content/explore-detection-vocal/Tensorflow/workspace/saved \
    --pipeline_config_path /content/explore-detection-vocal/Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config

In [ ]:
!zip -r /content/explore-detection-vocal/Tensorflow/workspace/saved.zip /content/explore-detection-vocal/Tensorflow/workspace/saved

In [ ]:
! git add --all

In [ ]:
pwd

In [ ]:
! git add 